视频教程
https://www.bilibili.com/video/BV11m4y1e7Yn?p=9&spm_id_from=pageDriver&vd_source=69b9597080eefd2cce7fd63de0c25e83

In [ ]:
from elasticsearch_dsl import connections, Index

In [10]:
connection = connections.create_connection(hosts=["http://localhost:9200"])

In [11]:
index = Index('books')

In [12]:
index.create()

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'books'})

In [13]:
index.exists()

HeadApiResponse(True)

In [ ]:
# index.delete()

In [14]:
from elasticsearch_dsl import Document, Text, Keyword, Integer, Boolean


class Books(Document):
    title = Text()
    description = Text()
    category = Keyword()
    price = Integer()
    in_stock = Boolean()

    class Index:
        name = "books"

In [19]:
import pandas as pd

df_books = pd.read_csv('./data/books.csv', encoding='gbk')
df_books

,title,description,category,price,in_stock
0,西游记,由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。,小说,10,False
1,红楼梦,由清朝作家曹雪芹创作，被誉为中国古代小说的巅峰之作。,小说,20,True


In [20]:
data = df_books.iloc[0, :]
print(data)
book = Books(title=data.title, description=data.description, category=data.category, price=data.price, in_stock=data.in_stock)

title                                                西游记
description    由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。
category                                              小说
price                                                 10
in_stock                                           False
Name: 0, dtype: object


In [21]:
for row_no, row in df_books.iterrows():
    book = Books(title=row.title, description=row.description, category=row.category, price=row.price, in_stock=row.in_stock)
    book.save()

In [22]:
from elasticsearch_dsl import Search, Q, connections

In [23]:
search = Search(index='books')
query = Q('match', title='西游')
s = search.query(query)
response = s.execute()

In [24]:
for hit in response.hits:
    print(f'title: {hit.title}')
    print(f'description: {hit.description}')

title: 西游记
description: 由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。


In [25]:
from elasticsearch_dsl import MultiSearch, Search

In [26]:
ms = MultiSearch(index='books')

s1 = Search()
s1 = s1.query('match', title='西游')
ms = ms.add(s1)

s2 = Search()
s2 = s2.query('match', title='梦')
ms = ms.add(s2)

In [27]:
responses = ms.execute()

for response in responses:
    for hit in response.hits:
        print(f'title: {hit.title}')

title: 西游记
title: 红楼梦


In [31]:
from util import search_query
from elasticsearch_dsl.query import Match

search = Search(index='books')
query = Match(title='西记')
search_query(search, query, columns=['title'])

title: 西游记


In [33]:
from elasticsearch_dsl.query import MatchAll

query = MatchAll()
search_query(search, query, columns=['title'])

title: 西游记
title: 红楼梦


In [34]:
from elasticsearch_dsl.query import MultiMatch

query = MultiMatch(query='西游记', fields=['title'], minimum_should_match=2)
search_query(search, query, columns=['title', 'description'])

title: 西游记
description: 由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。


In [39]:
from elasticsearch_dsl.query import Fuzzy

query = Fuzzy(title={'value': '东', 'fuzziness': 1})
search_query(search, query, columns=['title', 'description'])

title: 西游记
description: 由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。
title: 红楼梦
description: 由清朝作家曹雪芹创作，被誉为中国古代小说的巅峰之作。


In [41]:
from util import search_term
from elasticsearch_dsl.query import Range

term = Range(price={"gte": 20, "lte": 100})
search_term(search, term, columns=['title', 'price'])

title: 红楼梦
price: 20
